In [330]:
import pandas as pd

In [331]:
df = pd.read_csv("Datasets/customer_support_tickets.csv")
# df.head()
# print(df.columns.tolist())
# df.isnull().sum()


In [ ]:
# #Drop unnecessary columns right away
# df = df.drop([
#     'Ticket ID', 'Customer Name', 'Customer Email', 
#     'Customer Gender', 'Ticket Status', 'Resolution', 
#     'First Response Time', 'Time to Resolution', 
#     'Customer Satisfaction Rating'
# ], axis=1)

df["Ticket Type"].value_counts()
# print(df.columns.tolist())
# df["Product Purchased"].unique().tolist()

Ticket Type
Refund request          1752
Technical issue         1747
Cancellation request    1695
Product inquiry         1641
Billing inquiry         1634
Name: count, dtype: int64

# I am using Random Forest, XGBoost that is why using frequence encoding 
# As using frequence encoding, it losing info of Products as if TV and MObile have same frequence 200, it will store one time 200
# I have to use label encoding for as model is Random forest

In [285]:
df.columns.tolist()

['Customer Age',
 'Product Purchased',
 'Date of Purchase',
 'Ticket Type',
 'Ticket Subject',
 'Ticket Description',
 'Ticket Priority',
 'Ticket Channel']

In [333]:
# len(df['Product Purchased'].unique().tolist())
# Creates separate binary columns for each product
product_dummies = pd.get_dummies(df['Product Purchased'], prefix='Product')
df = pd.concat([df, product_dummies], axis=1)

df = df.astype({col: int for col in df.select_dtypes('bool').columns})


# Delete Product Purchased orginal col
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8469 entries, 0 to 8468
Data columns (total 50 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   Customer Age                            8469 non-null   int64 
 1   Product Purchased                       8469 non-null   object
 2   Date of Purchase                        8469 non-null   object
 3   Ticket Type                             8469 non-null   object
 4   Ticket Subject                          8469 non-null   object
 5   Ticket Description                      8469 non-null   object
 6   Ticket Priority                         8469 non-null   object
 7   Ticket Channel                          8469 non-null   object
 8   Product_Adobe Photoshop                 8469 non-null   int32 
 9   Product_Amazon Echo                     8469 non-null   int32 
 10  Product_Amazon Kindle                   8469 non-null   int32 
 11  Prod

In [ ]:
# df['Date of Purchase'].unique().tolist()
# df['Date of Purchase'] = pd.to_datetime(df['Date of Purchase'])
# df['Purchase_Year'] = df['Date of Purchase'].dt.year
# df['Purchase_Month'] = df['Date of Purchase'].dt.month
# df['Purchase_Day'] = df['Date of Purchase'].dt.day
# df['Purchase_DayOfWeek'] = df['Date of Purchase'].dt.dayofweek   # 0=Monday, 6=Sunday
# df['Purchase_Quarter'] = df['Date of Purchase'].dt.quarter

# # delete orginal col Date of Purchase

In [334]:
df['Ticket Subject'].unique().tolist()
# Combine Subject and Description
df['Combined_Text'] = df['Ticket Subject'].astype(str) + ' ' + df['Ticket Description'].astype(str)



In [335]:
df['Ticket Priority'].unique().tolist()
priority_mapping = {'Low': 1, 'Medium': 2, 'High': 3, 'Critical': 4}
df['Ticket Priority Mapped'] = df['Ticket Priority'].map(priority_mapping)
df['Ticket Priority Mapped'].unique().tolist()
# delete orginal col Ticket Priority

[4, 1, 3, 2]

In [336]:
df['Ticket Channel'].unique().tolist()
channel_dummies = pd.get_dummies(df['Ticket Channel'], prefix='Channel')
df = pd.concat([df, channel_dummies], axis=1)

# delete orginal col Ticket Channel


In [337]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download NLTK resources (only once)
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # 1. Lowercase
    text = text.lower()
    
    # 2. Remove punctuation / special characters
    text = re.sub(r'[^a-z\s]', '', text)
    
    # 3. Remove stopwords
    words = [word for word in text.split() if word not in stop_words]
    
    # 4. Lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]
    
    # Join words back to string
    return ' '.join(words)

# Apply to your column
df['Combined_Text_Cleaned'] = df['Combined_Text'].apply(preprocess_text)


[nltk_data] Downloading package stopwords to C:\Users\Faisal
[nltk_data]     Zamir\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Faisal
[nltk_data]     Zamir\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [338]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re

# CRITICAL FIX: Replace product placeholder with actual product names
def fix_product_placeholders(text, product_name):
    text = str(text)
    # Replace {product_purchased} with actual product name
    text = re.sub(r'\{product_purchased\}', product_name, text)
    text = re.sub(r'productpurchased', product_name, text)
    text = re.sub(r'product_purchased', product_name, text)
    return text

# Apply the fix - you need the original product names
# Assuming you have 'Product Purchased' column with actual product names
df['Combined_Text_Cleaned'] = df.apply(
    lambda row: fix_product_placeholders(row['Combined_Text_Cleaned'], row['Product Purchased']), 
    axis=1
)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
# Extract ONLY the meaningful parts from your messy text
def extract_meaningful_text(text):
    text = str(text)
    
    # Keep only sentences that contain actual problem descriptions
    meaningful_patterns = [
        r'not turning on',
        r'not working',
        r'crash',
        r'error',
        r'bug',
        r'issue',
        r'problem',
        r'not connecting',
        r'password',
        r'login',
        r'refund',
        r'billing',
        r'payment'
    ]
    
    sentences = text.split('.')
    meaningful_sentences = []
    
    for sentence in sentences:
        if any(pattern in sentence.lower() for pattern in meaningful_patterns):
            meaningful_sentences.append(sentence)
    
    return '. '.join(meaningful_sentences) if meaningful_sentences else 'no_meaningful_text'

# Apply extraction
df['Combined_Text_Cleaned'] = df['Combined_Text_Cleaned'].apply(extract_meaningful_text)

# TF-IDF only on meaningful content
tfidf = TfidfVectorizer(
    max_features=1000,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.7,
    stop_words='english'
)

X_text = tfidf.fit_transform(df['Combined_Text_Cleaned'])
print(f"Meaningful TF-IDF shape: {X_text.shape}")
print(f"Empty texts: {(df['Combined_Text_Cleaned'] == 'no_meaningful_text').sum()}")

KeyError: 'Meaningful_Text'

# Remove unncessary cols (see above)
# Please now handle target column Ticket Type
# then seperate X and Y

In [342]:
# #Drop unnecessary columns right away
# df = df.drop([
# 'Date of Purchase',
# 'Ticket Subject',
# 'Ticket Priority',
# 'Ticket Channel',
# 'Ticket Description',
# 'Combined_Text',
# 'Product Purchased',
# ], axis=1)


In [343]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
y = le.fit_transform(df['Ticket Type'])

df['Ticket Type'] = y
df['Ticket Type'].unique().tolist()


[4, 0, 1, 2, 3]

In [344]:
df['Combined_Text_Cleaned'].head(10)


0    product setup im issue GoPro Hero please assis...
1    peripheral compatibility im issue LG Smart TV ...
2    network problem im facing problem Dell XPS Del...
3    account access im issue Microsoft Office pleas...
4    data loss im issue Autodesk AutoCAD please ass...
5    payment issue im facing problem Microsoft Offi...
6    refund request im unable access Microsoft Surf...
7    battery life im issue Philips Hue Lights pleas...
8    installation support im issue Fitbit Versa Sma...
9    payment issue Dyson Vacuum Cleaner making stra...
Name: Combined_Text_Cleaned, dtype: object

In [352]:
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler

# Drop text columns and target column
X_others = df.drop(['Combined_Text_Cleaned', 'Ticket Type'], axis=1)
X_others.head()

# Convert all boolean columns to integers (True → 1, False → 0)
X_others = X_others.astype({col: 'int' for col in X_others.select_dtypes('bool').columns})

scaler = StandardScaler(with_mean=False)
X_scaled = scaler.fit_transform(X_others)

# # # Combine TF-IDF features with other numeric features
X_final = hstack([X_text, X_scaled])



# print(X_final.shape, y.shape)

# print(X_text.shape)
# print(tfidf.get_feature_names_out()[:20])

# X_others.dtypes.value_counts()


In [353]:
from sklearn.model_selection import train_test_split

# Split data (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)


In [354]:
from sklearn.ensemble import RandomForestClassifier


# Random Forest usually works well with mixed data types
model = RandomForestClassifier(n_estimators=100, random_state=42)

model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [355]:
y_pred = model.predict(X_test)


In [356]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.2089728453364817

Classification Report:
               precision    recall  f1-score   support

           0       0.21      0.17      0.19       357
           1       0.19      0.21      0.20       327
           2       0.21      0.22      0.21       316
           3       0.22      0.25      0.23       345
           4       0.21      0.20      0.20       349

    accuracy                           0.21      1694
   macro avg       0.21      0.21      0.21      1694
weighted avg       0.21      0.21      0.21      1694


Confusion Matrix:
 [[62 82 73 83 57]
 [53 70 58 85 61]
 [53 65 68 54 76]
 [60 75 58 85 67]
 [61 75 62 82 69]]
